In [ ]:
# 01 - Create databases of RRSG/qMRSG Challenge data

## Download data using shell

In [ ]:
from make_pooled_datasets import *
make_pooled_dataset('configs/3T_human_T1maps.json', '3T_human_T1maps')

## Imports

In [ ]:
from pathlib import Path
import pandas as pd
import json
import nibabel as nib
import numpy as np

from src.database import *

In [ ]:
configFile = Path('configs/3T_human_T1maps.json')
data_folder_name = '3T_human_T1maps'
output_filename = data_folder_name + '_database'

In [ ]:
df = create_database(configFile, data_folder_name)

## Save database

In [ ]:
df.to_excel(str(Path('databases/') / (output_filename + '.xlsx'))) # To excel
df.to_pickle(str(Path('databases/') / (output_filename + '.pkl'))) # To pickle (Python object)
df.to_csv(str(Path('databases/') / (output_filename + '.csv'))) # To csv

## Database manipulation demo

### View entire database

In [ ]:
df

### View a column values for a specific database ID (i.e. row)

In [ ]:
df.loc[1.001]

### Get the value of a cell entry (row/column intersection)

In [ ]:
df.loc[1.001]['T1 - genu (WM)']